# Exercice 1 - Simple 2D model

This first exercise will present the basic capabilities of ArchPy (how to define the stratigraphic pile, how to import boreholes and how to visualize the data and models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
import geone
import geone.covModel as gcm
import pyvista as pv
import ArchPy

## Building the model

Let's see step by step how to build an ArchPy model.

### Arch table object

The very first step is to instantiate an Arch_table object. This is the main object of ArchPy that contains all the useful functions (preprocessing, computing, plotting, etc.). It can be seen as the project.

The syntax is pretty simple: ``ArchPy.base.Arch_table(name = ..., seed = ..., verbose = ...)``.

- ``name`` corresponds to the project name and will be used as an identifier
- ``seed`` is the seed for the reproducibility of the stochastic events
- ``verbose`` is the level of information that will be printed by ArchPy, 0 for quasi-None and 1 for everything.

In [ ]:
T1 = ...

### Grid

Next we can define a simulation grid. In ArchPy, grids are simple regular cartesian grids and only required three inputs:
- ``dimensions`` (nx, ny, nz), the number of cells in three directions
- ``spacing`` (sx, sy, sz), the size of the cells in each directions
- ``origin`` (x0, y0, z0), the origin of the grid

For this exercise, let us a define a 2D slice grid of nx = 200 and nz = 100 with a spacing of sx = 0.5 and sz = 0.15. The origin is x0 = 0, y0 = 0 and z0 = -15.

The grid can be added to the arch table with the ``add_grid()`` method.

In [ ]:
# grid

sx = ...
sy = ...
sz = ...
x0 = ...
y0 = ...
z0 = ...
nx = ...
ny = ...
nz = ...

dimensions = (nx, ny, nz)
spacing = (sx, sy, sz)
origin = (x0, y0, z0)  

## add grid to the arch table
...

### Units and surfaces

Let us now create three stratigraphic units: C, B and A (C is above B which is above A). 

This is done with the classes ArchPy.base.Unit and ArchPy.base.Surface

**For each unit, a surface must be defined.** This surface delimits the top of the unit.

A surface is caracterized by : 
- contact type (onlap or erode), which influences how it interacts with other surfaces - Surface dictionnary (dic_surf) wich includes all parameters and method interpolations
- int_method : interpolation method --> grf, grf_ineq, MPS, kriging, ... 
- covmodel : covariance model to use if int_method is grf, grf_ineq, kriging
- mean : mean elevation of the surface
- etc.

A unit is caracterized by : 
- a name
- an order that defines the unit position in the pile, ranging from 1 (youngest unit) to n (oldest unit).
- a surface object
- Unit dictionnary (dic_facies), more info below ! 

In [ ]:
# units
#units covmodel
covmodelB = gcm.CovModel2D(elem=[('cubic', {'w':.5, 'r':[40, 40]})]) 
covmodelA = gcm.CovModel2D(elem=[('cubic', {'w':7, 'r':[30, 30]})])


# create Units 

# C unit as example
surf_C = ArchPy.base.Surface(contact = "onlap")
C = ArchPy.base.Unit(name = "C", order = 1, ID = 1, color = "lightblue",
                     surface = surf_C)


# Do not forget to add the covmodels to surface of B and A !

dic_s_B = {"int_method" : ... ,"covmodel" : ...}
surf_B = ...  # create a surface object
B = ArchPy.base.Unit(name = ..., order = ..., ID = ..., color="gold",
                     surface = ...)


dic_s_A = ...
surf_A = ...  
A = ...

### Pile

Finally, the units are added to a Pile object with the ``add_unit`` method of the Pile.

In [ ]:
P1 = ...  # create a pile object
...  # add units

T1.set_Pile_master(P1)# define the main pile

### Boreholes

Let's see now how to import boreholes into the model.

For this ArchPy requires three different data text files.: a list of borholes, a list of unit data and a list of facies data.

They can have any extensions but the recommended are :

list of boreholes -> .lbh

list of unit data -> .ud

list of facies data -> .fd

For this example, the files are in the 2_data_folder.

Let's detail these files :
- .lbh : text file with five columns listing all the boreholes in the data. It has five columns and default headers are :
    - bh_ID : borehole identifier to know at which borehole a unit/facies data belongs
    - bh_x : x borehole coordinate
    - bh_y : y borehole coordinate
    - bh_z : z borehole coordinate
    - bh_depth : borehole depth


- .ud : text file with four columns listing all the stratigraphical unit interval data. Default headers are :
     - bh_ID : borehole identifier to know at whcih borehole the unit data belongs
     - Strat : Unit identifier to know at which unit this unit interval data belongs
     - top : top elevation of the interval
     - bot : bot elevation of the interval
    
        
- .fd : text file with four columns listing all the facies interval data. Default headers are :
     - bh_ID : borehole identifier to know at whcih borehole the unit data belongs
     - facies_ID : facies identifier to know at which facies this facies interval data belongs
     - top : top elevation of the interval
     - bot : bot elevation of the interval


First --> we have to import these files with pandas 

In [ ]:
import pandas as pd

list_bhs = ...  # import list of boreholes file
unit_data = ...  # import unit data file
facies_data = ...  # import facies data file

We can now import the boreholes with the following functions. I'm not going into the details but the first one create a uniform and clean geological base from the unit and facies data file. And the second create the borehole object for ArchPy.


Note that We don't import the facies information here (``extract_facies = False``).

In [ ]:
db, lbhs = ArchPy.inputs.load_bh_files(list_bhs, facies_data, unit_data)
boreholes = ArchPy.inputs.extract_bhs(db, lbhs, T1, extract_facies=False, vb=1)

Let us add these boreholes to the table with ``add_bh()`` method

In [ ]:
...

We can see the boreholes with ``plot_bhs()`` method

In [ ]:
T1.plot_bhs()

We still need to process these boreholes, i.e. extract the information for the simulations. fortunately, this is done automatically by archpy by simply using the ``process_bhs()`` method !

In [ ]:
T1.process_bhs()

### Simulations

with the following command:

- ``compute_surf(nreal)``

Let's consider a total of 10 unit realizations (nreal = 10)

In [ ]:
T1.compute_surf(...)

In [ ]:
## plots

plt.figure(figsize=(10, 6), dpi=200)
for i in range(6):
    
    plt.subplot(3, 2, i+1)
    plt.imshow(T1.get_units_domains_realizations(i, fill="color")[:, 0], origin="lower", aspect=.5)
    plt.title("Realization {}".format(i))
    

## Testing with erode

This is nice but we can now test different options of ArchPy, for example the choice between onlap and erode of the surface. What happens if we modify the top surface of B to ``erode`` ?


In [ ]:
## modify the contact of the surface of B


In [ ]:
# reprocess the boreholes  --> pile has changed
T1.bhs_processed = 0  # this line reset the flag that indicate that the boreholes are processed
T1.process_bhs()

In [ ]:
# re run simulations
T1.compute_surf(50)

# plots
plt.figure(figsize=(10, 6), dpi=200)
for i in range(6):
    
    plt.subplot(3, 2, i+1)
    plt.imshow(T1.get_units_domains_realizations(i, fill="color")[:, 0], origin="lower", aspect=.5)
    plt.title("Realization {}".format(i))


In [ ]:
# 3D plot with boreholes
p = pv.Plotter()  # create a pyvista plotter...

T1.plot_units(0, plotter=p)  # put plotter in archpy plotting function...
T1.plot_bhs(plotter=p)  # put plotter in archpy plotting function...

p.show()  # ... and display !

ArchPy also offers some functionnalities to deal with uncertainties and probabilities.

Try the ``plot_proba()`` method

In [ ]:
# plot probability
...

## Facies

Facies are easily integrated with ``ArchPy.base.Facies``. A facies can then be passed to the unit with the add_facies method. Warning, this method is an "adding" method it does not remove previously added facies. They should be remove manually wiht the following code : unit.list_facies = [].

As a remainder, C unit is uniformly composed of silts, B unit is composed of sand and gravel and A of silts and clay.

In [ ]:
# Create facies
sand = ...
...

# add facies
...

### Modify pile

Now, we have to modify the pile to indicate to ArchPy how to simulate these facies. We will use simple Sequential Indicator Simulations (**SIS**) in this exemple. SIS is on of the most simple geostatistical methods when it comes to facies modeling as it does not require many inputs. Indeed, it requires at least one covariance model for each unit. 
For simplicity of the example, these are given below (covmodel_SIS_A and covmodel_SIS_B). They represent the spatial variability of the facies that will be simulated.

Each facies methods have specific modeling parameters that are given to ArchPy by the use of a dictionnary (dic_facies). This dictionnary can have multiple entries such as:
   - f_method : filling method (SIS, MPS, homogenous, SubPile or TPGs) 
   - f_covmodel : facies covmodels to use with the SIS
   - probability : facies proportions, in the same order than the facies passed to the Unit.
   - (see ArchPy documentation for all the parameters)
  
 
Create these dictionnaries and add them to the corresponding units. There is no need to consider the proportions for this exercise.

In [ ]:
# covariance models
covmodel_SIS_A = gcm.CovModel3D(elem = [("exponential", {"w":.25,"r":[70, 70, 5]})], alpha=0, name="vario_SIS") # input variogram
covmodel_SIS_B = gcm.CovModel3D(elem = [("exponential", {"w":.25,"r":[50, 50, 5]})], alpha=0, name="vario_SIS") # input variogram

dic_f_B = ...
dic_f_A = ...

# set dictionnaries
B.set_dic_facies(dic_f_B)
A.set_dic_facies(dic_f_A)

### Re-add boreholes with facies this time

We need to reimport the boreholes with the facies information this time !

In [ ]:
T1.rem_all_bhs()  # remove all boreholes

In [ ]:
boreholes = ArchPy.inputs.extract_bhs(db, lbhs, T1, extract_facies=True, vb=1)  # extract boreholes from database
T1.add_bh(boreholes)

### Reprocess and Rerun

Let's run again the simulations. This time 5 unit realizations and 3 facies realizations.

``T1.compute_facies(nreal)``

In [ ]:
T1.process_bhs()  # process again because there is new boreholes

# simulations
...

In [ ]:
# plot
plt.figure(figsize=(5, 5.5), dpi=150)
for iu in range(2):
    for ifa in range(3):
    
        plt.subplot(3, 2, iu*3+ifa+1)
        plt.imshow(T1.get_facies(iu, ifa, all_data=False)[:, 0], origin="lower")
        plt.title("un {} / fa {}".format(iu, ifa))

In [ ]:
p = pv.Plotter()

T1.plot_facies(0, 1, plotter=p)
T1.plot_bhs("facies", plotter=p)
p.show()

## Property

Let's now add a phyiscal property: *log Hydraulic conductivity*. For recall, we assume a mean value of -3.5 for sand, -2 for gravel, -8 for clay and -6.5 for silt. 

We will create a property object with ``ArchPy.base.Prop``.

Important: Property objects are directly added to the project and not to the facies.

Arguments are : 
- name 
- facies (list, the facies in which to simulate the prop)
- covmodels (list, covariance models for the simulation, one for each facies, same order of facies) 
- means (list, mean values for the simulation, one for each facies)
- int_method (list, grf method --> SGS or FFT), which method to use to interpolate.
- x (position of the hard data, if any)
- v (values of the hard data, if any)
- def_mean (default value to use if a facies have not been added to the "facies" arguments
- vmin, vmax (min and max values for the properties, simulated values below (resp. above) will be capped.

In this example one property is defined for the four facies, each with a different covariance models given in list_covmodels, a list of 3D covmodels objects (see geone doc.). The order of the covmodels must be consistant with the order of the facies in list_facies. The same applies for the means parameter.

Finally the property is added to the project with the method ``Arch_table.add_project()``

In [ ]:
# covmodels for the property model
covmodelK = gcm.CovModel3D(elem=[("exponential",{"w":0.3,"r":[5,5,1]})], name="K_vario")

K = ArchPy.base.Prop(name = ...,
                     facies = [..., ..., ..., ...],
                     covmodels = [..., ..., ..., ...],
                     means = [..., ..., ..., ...],
                     int_method = [..., ..., ..., ...])

# add prop to table


And run simulations !

Set ``nreal = 2`` to start with

In [ ]:
T1.compute_prop(2)

In [ ]:
# Plots

plt.figure(figsize=(5, 7), dpi=150)
o = 1
for iu in range(2):
    for ifa in range(3):
        for ip in range(1):
            
            plt.subplot(4, 2, o)
            plt.imshow(T1.getprop("K", iu, ifa, ip, all_data=False)[:, 0], origin="lower")
            plt.title("un {} / fa {} / pr {}".format(iu, ifa, ip))
            
            o += 1
            

In [ ]:
#plot 3D
T1.plot_prop("K")

In [ ]:
# plot the mean
T1.plot_mean_prop("K")